# 2/09/2025 Deep Learning exam implementation
## Diego Meloni 536041

# 1: Architecture choice
I chose a Hopfield Network as a local optimizer to solve the given task, since the problem is structured as a graph where nodes only have partial labeling, but do not contain any additional features or information. If nodes had features, an architecture such as a Graph Convolutional Network would have been more appropriate.
A regular MLP (Multi layer perceptron) would not be a good architecture choice since it would not be able to exploit nodes similarity.

The implementation matches the solution I provided in the exam, so no CHANGE blocks were required.


# Notebook setup and data download
Here I download the dataset and setup the notebook.

In [12]:
import pickle as pk
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
import requests

# Dataset parts
parts = [
    "https://github.com/Diego-Meloni/AI-Projects/raw/refs/heads/main/Hopfield%20Network/input_data_part01.pkl",
    "https://github.com/Diego-Meloni/AI-Projects/raw/refs/heads/main/Hopfield%20Network/input_data_part02.pkl",
    "https://github.com/Diego-Meloni/AI-Projects/raw/refs/heads/main/Hopfield%20Network/input_data_part03.pkl",
    "https://github.com/Diego-Meloni/AI-Projects/raw/refs/heads/main/Hopfield%20Network/input_data_part04.pkl",
    "https://github.com/Diego-Meloni/AI-Projects/raw/refs/heads/main/Hopfield%20Network/input_data_part05.pkl"
]

output_file = "input_data.pkl"

# Function to download the dataset (first time only)
if not os.path.exists(output_file):
    with open(output_file, "wb") as f_out:
        for idx, url in enumerate(parts):
            print(f"Downloading part {idx+1}...")
            r = requests.get(url)
            r.raise_for_status()
            f_out.write(r.content)
    print("Download completed.")

else:
    print("File already present, download skipped.")

# Load the dataset
with open(output_file, "rb") as f:
    data_dict = pk.load(f)

# Weight matrix, initial labels and true labels are provided
W = data_dict['similarity_matrix']
labels = data_dict['labels']
true_labels = data_dict['true_labels']

# Check if shapes are correct
print(f"\nWeight matrix shape: {W.shape}")
print(f"Labels shape: {labels.shape}")
print(f"True labels shape: {true_labels.shape}")

Download completed.

Weight matrix shape: (5000, 5000)
Labels shape: (5000,)
True labels shape: (5000,)


# 2: Preprocessing
I considered two possible preprocessing steps:
- Mapping string labels into numeric values {1, 0, -1}. (In this dataset the labels were already numeric, but I implemented a custom function to handle the string case if needed.)
- Setting the diagonal of the weight matrix to 0, in order to prevent self-connections between neurons.


In [13]:
# Maps the input labels to {-1, 0, 1} values or returns an error.
def preprocess_labels(labels):
    # Case 1: labels are strings
    if isinstance(labels[0], str):
        new_labels = []
        for l in labels:
            if l == "Database":
                new_labels.append(1)
            elif l == "Unknown":
                new_labels.append(0)
            else:
                new_labels.append(-1)
        return np.array(new_labels)

    # Case 2: labels are numbers
    elif isinstance(labels[0], (int, float, np.int64)):
        ok_values = [-1, 0, 1]
        if all(l in ok_values for l in labels):
            return labels.astype(int)
        else:
            print("Input format incorrect: numeric values not in {-1,0,1}")
            return None

    # Case 3: wrong type
    else:
        print("Input format incorrect: labels must be strings or {-1,0,1}")

In [14]:
# Apply the preprocessing steps
labels = preprocess_labels(labels) # Maps the input labels to {-1, 0, 1} values if needed.
np.fill_diagonal(W, 0)  # Removes eventual self-connections in the neurons.

# 3a, 3b, 3c: Implementation of the Hopfield Network as Local Optimizer
In line with my exam solution, my implementation takes as input:
- The number of neurons of the Hopfield Network (size).
- The weight matrix.
- The partial labels of the neurons.
- The maximum number of iterations (default value set to 100, but modifiable at call time).

The model separates the neurons into two categories:
- **Fixed neurons**: initially labeled with +1 or -1. They are not updated during the dynamics, but their influence contributes to the updates of the initially unlabeled neurons.
- **Free neurons**: initially labeled with 0. Their states are updated asynchronously, meaning neuron **i** is updated using the most recent values of neurons 0 → i-1 and the old values of neurons i+1 → n. Each update considers the influence of fixed neurons (bias) and the weighted input from other neurons.

The model stops either when it reaches the maximum number of iterations, or when it reaches an equilibrium state (no updates occur running the dynamic).


## NOTE
The implementation follows my exam solution exactly. I chose to **add** the bias rather than subtract it, because we want to add the influence of the fixed neurons instead of subtracting it.

Since the bias can take both positive and negative values, it still behaves as a threshold. I have also noticed the dynamics are more effective when the bias is added, leading to better and more consistent results (also confirmed by additional tests I carried out separately).

# 3e: Hyperparameter tuning and selection
The model has only two hyperparameters:
- The number of neurons
- The maximum number of epochs

As stated in the exam, hyperparameter tuning is not required in this case. The number of neurons is fixed by the graph structure, and the dynamics usually converge in just a few iterations, making epoch tuning unnecessary.

I implemented two stopping conditions: one when an equilibrium state is reached (no neuron is updated), and one when the maximum number of epochs is reached.


In [15]:
import numpy as np
# Hopfield network as local optimizer
class HopfieldNetworkLO:
    def __init__(self, size,  weight_matrix, labels, max_iterations=100):
        """Initialize network with a given weight matrix."""
        self.W = np.array(weight_matrix)  # Convert weight matrix to numpy array
        self.N = self.W.shape[0]  # Number of neurons
        # Checks if the declared number of neurons is coherent with Weight matrix dimension
        if self.N != size:
          print('Declared number of neurons and Weight matrix shape not corresponding')
        self.max_iterations = max_iterations

        """ Divide the neurons in 2 categories based on their initial label"""
        self.state = np.array(labels, dtype=int)  # Convert labels to an integers array
        self.free_neurons = np.where(self.state == 0)[0]  # Indices of neurons to update
        self.fixed_neurons = np.where(self.state != 0)[0]  # Indices of fixed neurons

    @property # To call the method without () since there are no parameters.
    def run_dynamics(self):
        """Perform asynchronous updates for neurons labeled as '0'."""
        # Compute external influence from fixed neurons
        b = np.dot(self.W[:, self.fixed_neurons], self.state[self.fixed_neurons])  # Activation threshold
        e = 0 # Counter for epochs

        for e in range(self.max_iterations):
            e += 1
            updated = False
            for i in self.free_neurons:
                # Compute total input sum (including fixed neurons' influence)
                activation = np.dot(self.W[i], self.state) + b[i]
                new_state = 1 if activation >= 0 else -1

                if new_state != self.state[i]:
                    self.state[i] = new_state
                    updated = True

            # Stop if no changes occur (equilibrium state)
            if not updated:
                print(f"Number of epochs: {e}")
                return self.state

        # Stop if the maximum number of epochs is reached
        print("Maximum number of epochs reached")
        return self.state

# 4: Evaluation function
I created a function which evaluates the reconstruction accuracy of the initially unlabeled neurons.

It takes as input the initial partial labels, our prediction, and the true labels, and returns:
- The accuracy of the model (correct predictions/total predictions).
- The list of predictions.
- The list of true labels corresponding to the prediction.
- The position of the unknown neurons in the original labels.

In [16]:
# Function to compute accuracy only on the unknown (0-labelled) authors
def evaluate_accuracy_on_unknowns(initial_labels, inferred_labels, true_labels):
    # Find indices of neurons that were initially unknown
    unknown_indices = np.where(initial_labels == 0)[0]

    # Predicted vs true for only these indices
    preds = inferred_labels[unknown_indices]
    trues = true_labels[unknown_indices]

    # Compute accuracy
    accuracy = np.mean(preds == trues)
    print(f"\nAccuracy on unknown (0-labelled) nodes: {accuracy}")
    return accuracy, preds, trues, unknown_indices

# Results
I instantiated the Hopfield Network and applied it to the input graph in order to update the labels of the unknown authors (initial label = 0). The model’s performance is then evaluated by measuring its accuracy.  

For clarity, I also included visualizations of both the dataset and the predictions, together with a sample of the reconstructed labels produced by the dynamics.


In [17]:
# Initialize and update the network
size = int(labels.shape[0]) # Number of neurons of the Network
hopfield_net = HopfieldNetworkLO(size, W, labels)
inferred_state = hopfield_net.run_dynamics

# Visualize our original dataset, the predictions and the true labels
print(f"\nOriginal labels: {labels}")
print(f"Inferred labels: {inferred_state}")
print(f"True labels:     {true_labels}")

# Evaluate
acc, preds, trues, unknown_indices = evaluate_accuracy_on_unknowns(labels, inferred_state, true_labels)
print(f"Number of predictions: {int(preds.shape[0])}")

# Print first 20 unknowns with predicted vs true
print("\nFirst 20 predictions for initially unknown authors:")
for i in range(min(20, len(unknown_indices))):
    idx = unknown_indices[i]
    print(f"  Author {idx+1}: predicted= {preds[i]}, true= {trues[i]}")

Number of epochs: 3

Original labels: [-1  0  1 ...  1 -1 -1]
Inferred labels: [-1 -1  1 ...  1 -1 -1]
True labels:     [-1 -1  1 ...  1 -1 -1]

Accuracy on unknown (0-labelled) nodes: 0.7986666666666666
Number of predictions: 750

First 20 predictions for initially unknown authors:
  Author 2: predicted= -1, true= -1
  Author 8: predicted= -1, true= -1
  Author 16: predicted= 1, true= 1
  Author 32: predicted= -1, true= -1
  Author 38: predicted= -1, true= -1
  Author 44: predicted= -1, true= -1
  Author 55: predicted= 1, true= 1
  Author 65: predicted= -1, true= -1
  Author 67: predicted= -1, true= -1
  Author 82: predicted= 1, true= 1
  Author 84: predicted= -1, true= 1
  Author 89: predicted= -1, true= -1
  Author 95: predicted= -1, true= -1
  Author 102: predicted= 1, true= 1
  Author 120: predicted= 1, true= -1
  Author 145: predicted= 1, true= 1
  Author 170: predicted= -1, true= -1
  Author 173: predicted= -1, true= -1
  Author 179: predicted= -1, true= -1
  Author 182: predict

The next block displays the confusion matrix.

The results show a fairly balanced matrix, moreover the model has a good performance on the predicted labels meaning it managed to effectively leverage the similarity of the authors.
This confirms Hopflied Network as Local Optimizer was a good architecture choice for our task.